# Output from Topic Modeling - basic model validated, no plots

### Using larger model picked by cross-validation: 7 topics (no transformation)

**How does this notebook work?**

To make it more readable, most of the code is hidden. If you need to see the code or change it, press on the link by `toggle on` at the cell location. While you can't see the code, simple instructions are still displayed above to describe what is happening.

Steps:

    * Load libraries
    * Note: need to install orca for inline interactive plots (frequency plot)
    * May need to download: nltk.download(['punkt', 'stopwords', 'wordnet'])
    
**Important:** to view all possible arguments local modules `helper.helper_funs`, `helper.word_preprocess` and `methods.LDAprep`. Precise definitions can be found there.

In [1]:
%matplotlib inline

from matplotlib import pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # may be a deprecation warning 

import pandas as pd
import numpy as np
from itertools import product

from os import path
import random
import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import stopwords

import gensim
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


# local modules
from helper.helper_funs import time_filename, save_folder_file, logged
from helper.word_preprocess import DataLoader, CleanText, WordCount, PlotWords
from methods.LDAprep import GensimPrep, GenMod, Diagnostics, ModelResults 

from helper.dk_ipython import toggled
from IPython.display import HTML
toggled()

## Set parameters:

`seed`: set seed number for reproducibility.

`start`, `max_num_topics`, `step` are for setting topic number (`int`) for cross-validation (lowest topic number, highest, step size)

`col` is for the columns with text. If more than one is chosen, text will be concatenated as one document per art center.

`stop` is to set the language to use for removing words with little meaning (the, a, are, etc.). french is included here because some French may be present even after removing observations labeled as French.

`stop.update` is for adding additional words to be remove, specific to this problem. For example, 'art' is not a meaningful word for our purposes. More words may be added by hand.

**To view the handpicked words removed, click on the `here` link by `toggle on/off`, under the first cell to review the code**

In [2]:
seed = 2018 
random.seed(seed)

# setting for tuning topic number in LDA
eval_every = 5      # the smaller number, the more precise the plot

col = ['Mandate/Mission', 
       'Main Community-Engaged Arts / Arts for Social Change Activities', 
       'Additional Info']  # name of comlumn(s) with text

stop = set(stopwords.words(['english', 'french'])) # words to remove from corpus

# stop.update([ 'community', 'use', 'make', 'organization', 'people', 'art', 'arts', 'artist', 'artists']) # hand picked words to remove



## Load and clean the data

Use the `DataLoader` to load, clean, remove accents, duplicates, empty rows, foreign languages and URLs. Small words (2 letters or less) are removed,then words are shortened and simplified with lemmatization and stemming. Stemming can be changed from `Porter` to `Lancaster` (more conservative) or `Snowball` (language specific).

Result: `processed_data` is a list of lists, containing every unique (stemmed and lemmatized) words in each document.
`pd_data` is a DataFrame with the raw text, for later display.

**Note that the path should be replaced as needed**

In [3]:
data = DataLoader(filename='artbridges_profiles.csv', 
                       data_folder='data',
                       colname=col,  # column(s) to remove weird chars
                       rm_NAs=False, # completely remove records with NaNs 
                                     #(implemented for 1st column only, other cols replace NaN with empty strings and concat)
                       removed_language=['french']) # list of languages to remove (lowercase)

donnee = data.data_loader()  # examples of problem mission statements: â, Lâ 


clean = CleanText(donnee,
                  stop=stop,
                  stemmer='Porter',
                  remove_urls=True)

processed_data = clean.preprocess()

pd_data = pd.DataFrame({'Text': donnee}) # for using unstemmed text later

Reading and cleaning data file...

Removing duplicates in column(s): ['Mandate/Mission', 'Main Community-Engaged Arts / Arts for Social Change Activities', 'Additional Info']...

Removing french words...

Done!


Only plain letters kept and words of 3 letters or more.

63 url(s) removed.


In [4]:
gp = GensimPrep(processed_data)
lda_dict = gp.gensimDict()
bow_rep = gp.gensimBOW(lda_dict)

Removing words of less than 3 characters, and words present in at least 0.8% of documents

Removing gaps in indices caused by preprocessing...

Saving gensim dictionary to corpus_data/2018-12-4/Gensim_dict_Params_MWL3_PD0.8__8h12m10s.dict

Saving .mm matrix to corpus_data/2018-12-4/BOWmat_8h12m10s.mm



## Running Optimized LDA Model 

__*using results from validation process*__

In [5]:
get_model = GenMod(bow_rep, lda_dict)

lda_model = get_model.LDA(eval_every=eval_every,
                          random_state=seed, num_topics= 7, alpha= 0.25, decay= 0.75)

Parameters used in model: 
Number of topics: 7
TFIDF transformation: False
Number of iterations: 10000
Batch size: 100
Update every 1 pass
Number of passes: 10
Topic inference per word: True
Alpha: 0.25
Eta: auto
Decay: 0.75
Minimum probability: 0.05
Minimum_phi_value: 0.02
Evaluate every: 5
Random seed: 2018

Saving LDA model to: 
saved_models/LDA/2018-12-4/LDA_Params_NT7_TFIDFFalsePer_word_topicTrue_8h12m17s.model


**Printing** linear combination of keywords for topics.

## Results: diagnostic plots and summary tables

## Tables with Topic Modeling Summary

Tables showing the distribution of words within topics and distribution of topics within documents (1 document = all texts from 1 organization)

**Important:** Topic are numbered starting at 0. For example, if we have 4 topics, they will be numbered from 0 to 3.

In [6]:
diag = Diagnostics(lda_dict,
                   bow_rep, 
                   processed_data)

warnings.filterwarnings("ignore", category=FutureWarning)

diag.LDAvis(lda_model)

Rendering visualization...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.003596  0.047642       1        1  49.471329
1     -0.063374 -0.069384       2        1   8.785878
2      0.030317 -0.005826       3        1   9.105540
3      0.040891 -0.049541       4        1   8.112377
4     -0.085337  0.039139       5        1   3.914356
5      0.038110  0.064690       6        1  13.595999
6      0.042989 -0.026720       7        1   7.014517, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
69    Default  618.000000       art  618.000000  30.0000  30.0000
263   Default  249.000000     youth  249.000000  29.0000  29.0000
71    Default  441.000000    commun  441.000000  28.0000  28.0000
276   Default   42.000000  aborigin   42.000000  27.0000  27.0000
37    Default  313.000000    artist  313.000000  26.0000  26.0000
93    Default  303.000000   program  303.000000  25.0000  25.0000
6     Default  127.000000    cultur  127.000000  24.0000  24.0000
72    Default  133.000000   creativ  133.000000  23.0000  23.0000
150   Default   54.000000    nation   54.000000  22.0000  22.0000
843   Default   24.000000     urban   24.000000  21.0000  21.0000
29    Default   78.000000     space   78.000000  20.0000  20.0000
119   Default   48.000000      danc   48.000000  19.0000  19.0000
98    Default   93.000000    school   93.000000  18.0000  18.0000
138   Default   42.000000   indigen   42.000000  17.0000  17.0000
170   Default  138.000000  workshop  138.000000  16.0000  16.0000
232   Default  105.000000     organ  105.000000  15.0000  15.0000
33    Default  179.000000      work  179.000000  14.0000  14.0000
7     Default   41.000000    disabl   41.000000  13.0000  13.0000
65    Default   19.000000     women   19.000000  12.0000  12.0000
128   Default   41.000000     first   41.000000  11.0000  11.0000
213   Default   46.000000     stori   46.000000  10.0000  10.0000
248   Default   42.000000      focu   42.000000   9.0000   9.0000
5     Default  115.000000     creat  115.000000   8.0000   8.0000
94    Default  114.000000   project  114.000000   7.0000   7.0000
21    Default  130.000000     peopl  130.000000   6.0000   6.0000
99    Default   35.000000    studio   35.000000   5.0000   5.0000
229   Default   57.000000      live   57.000000   4.0000   4.0000
586   Default   31.000000     write   31.000000   3.0000   3.0000
469   Default   42.000000     video   42.000000   2.0000   2.0000
107   Default   52.000000     share   52.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
586    Topic7   15.352469     write   31.577698   1.9360  -4.2416
417    Topic7    2.247307     green    4.790524   1.9003  -6.1631
771    Topic7    5.890595      know   12.645258   1.8933  -5.1995
271    Topic7    1.096508      rent    2.471091   1.8447  -6.8807
812    Topic7    5.623512   calgari   13.335541   1.7937  -5.2459
282    Topic7    6.272283    believ   18.101231   1.5973  -5.1367
123    Topic7    4.793195     everi   12.981709   1.6608  -5.4057
10     Topic7   12.205398    experi   52.673992   1.1949  -4.4710
72     Topic7   22.042568   creativ  133.657639   0.8549  -3.8799
170    Topic7   18.058121  workshop  138.398804   0.6206  -4.0793
71     Topic7   31.347918    commun  441.769073   0.0115  -3.5277
97     Topic7    5.038920      safe   15.603278   1.5269  -5.3557
69     Topic7   29.074217       art  618.109131  -0.3996  -3.6030
5      Topic7   11.893429     creat  115.731888   0.3819  -4.4969
107    Topic7    7.883559     share   52.744488   0.7565  -4.9081
536    Topic7    6.371038       one   30.966843   1.0760  -5.1211
93     Topic7   14.515869   program  303.435089  -0.3827  -4.2976
21     Topic7   10.364314     peopl  130.835907   0.1216  -4.6345
95     Topic7   10.817430    provid  155.256195  -0.0067  -4.5917
301    Topic7    7.567345   express   6

<hr>

### Table 1: Show Most Important Topic in each Document.

Displays the topic number that contributes most to document (in terms of marginal probability). Very high probability (close to 1) indicates that the topics are apparently easily distinguished from one another. 

Only 20 documents are displayed below, but all organizations with non-empty text are gathered in this table.

In [7]:
mod_res = ModelResults(lda_model, bow_rep, donnee)

df_dominant_topic = mod_res.format_topics_sentences()

small_dom_topic = pd.concat([df_dominant_topic.Dominant_Topic, 
                             df_dominant_topic.Percent_Contribution, 
                             df_dominant_topic.Important_Keywords,
                             df_dominant_topic.Mission], axis = 1)

small_dom_topic.columns = ['Dominant_Topic', 'Percent_Contribution', 'Important_Keywords', 'Mission']

small_dom_topic.head(15)

Saving the table to: results/2018-12-4/dominant_topic_per_text__8h12m31s.csv


,Dominant_Topic,Percent_Contribution,Important_Keywords,Mission
0,3.0,0.9592,"art, commun, space, stori, cultur, creativ, sh...",our goal is to have the space become self-sust...
1,0.0,0.5118,"art, commun, youth, artist, program, work, pro...",white water gallery is a not-for-profit artist...
2,0.0,0.3697,"art, commun, youth, artist, program, work, pro...",women sweetgrass film festival 2011
3,2.0,0.8110,"commun, art, artist, program, creativ, studio,...",(in development):\n\n -to provide appropriat...
4,0.0,0.8157,"art, commun, youth, artist, program, work, pro...",to create original physical theatre. to share ...
5,1.0,0.9887,"commun, art, nation, cultur, indigen, danc, wo...",the woodland cultural centre is a first nation...
6,0.0,0.5568,"art, commun, youth, artist, program, work, pro...",to provide an open and supportive environment ...
7,2.0,0.5737,"commun, art, artist, program, creativ, studio,...",the mandate of the 4cs foundation is to build ...
8,3.0,0.9694,"art, commun, space, stori, cultur, creativ, sh...","4elements living arts is a multidisciplinary, ..."
9,0.0,0.9757,"art, commun, youth, artist, program, work, pro...",7th generation image makers is a community art...


<hr>

### Table 2: Most Important Document/Text for each Topic

These are the documents that were most clealy classified into a given topic.

In [8]:
tab = mod_res.top_texts_per_topic(df_dominant_topic)

tab = pd.concat([tab.Dominant_Topic, 
                             tab.Percent_Contribution, 
                             tab.Important_Keywords,
                             tab.Mission], axis = 1)

tab.columns = ['Dominant_Topic', 'Percent_Contribution', 'Important_Keywords', 'Mission']
tab

Saving the table to: results/2018-12-4/top_texts_per_topic_8h12m32s.csv


,Dominant_Topic,Percent_Contribution,Important_Keywords,Mission
0,0.0,0.9934,"art, commun, youth, artist, program, work, pro...",gallery mandate: incorporated in 1988 and work...
1,1.0,0.9887,"commun, art, nation, cultur, indigen, danc, wo...",the woodland cultural centre is a first nation...
2,2.0,0.9902,"commun, art, artist, program, creativ, studio,...","founded in 1987, canadian stage is one of the ..."
3,3.0,0.9879,"art, commun, space, stori, cultur, creativ, sh...",the heroes of 107th project aims to:\n\n-empow...
4,4.0,0.9454,"aborigin, art, urban, youth, commun, artist, p...",mandate: urban society for aboriginal youth (u...
5,5.0,0.9865,"art, program, commun, school, artist, disabl, ...",in-definite arts is a nonprofit visual arts ce...
6,6.0,0.9705,"commun, art, creativ, women, workshop, write, ...","established in 1991 in calgary, alberta, canad..."


### Table 3: How documents are distributed between topics

In [9]:
top_dist = mod_res.topic_distribution(df_dominant_topic, tab)

top_dist

Saving the table to: results/2018-12-4/doc_distribution_in_topics_8h12m32s.csv


,Dominant_Topic,Important_Keywords,Num_Documents,Perc_Documents
0,0.0,"art, commun, youth, artist, program, work, pro...",129,0.56087
1,1.0,"commun, art, nation, cultur, indigen, danc, wo...",17,0.07391
2,2.0,"commun, art, artist, program, creativ, studio,...",15,0.06522
3,3.0,"art, commun, space, stori, cultur, creativ, sh...",16,0.06957
4,4.0,"aborigin, art, urban, youth, commun, artist, p...",7,0.03043
5,5.0,"art, program, commun, school, artist, disabl, ...",31,0.13478
6,6.0,"commun, art, creativ, women, workshop, write, ...",15,0.06522


<hr>
<hr>

## Update Corpus with BC Specific Data

Update the trained model with additional data from the BC dataset, using the `Mission/Mandate` and `Additional Info` columns.

In [10]:
col2 = ['Mandate/Mission', 
       'Additional Info']  

data2 = DataLoader(filename='artbridges_test.csv', 
                       data_folder='data',
                       colname=col2,  # column(s) to remove weird chars
                       rm_NAs=False, # completely remove records with NaNs 
                                     #(implemented for 1st column only, other cols replace NaN with empty strings and concat)
                       removed_language=['french']) # list of languages to remove (lowercase)

donnee2 = data2.data_loader()  # examples of problem mission statements: â, Lâ 


clean2 = CleanText(donnee2,
                  stop=stop,
                  stemmer='Porter',
                  remove_urls=True)

processed_data2 = clean2.preprocess()

pd_data2 = pd.DataFrame({'Text': donnee2}) # for using unstemmed text later

Reading and cleaning data file...

Removing duplicates in column(s): ['Mandate/Mission', 'Additional Info']...

Removing french words...

Done!


Only plain letters kept and words of 3 letters or more.

9 url(s) removed.


In [11]:
gp2 = GensimPrep(processed_data2)
lda_dict2 = gp2.gensimDict()
lda_id2 = lda_dict2.token2id
bow_rep2 = gp2.gensimBOW(lda_dict2)


# tfidfTrans2 = gp2.tfidf_trans(bow_rep2)

Removing words of less than 3 characters, and words present in at least 0.8% of documents

Removing gaps in indices caused by preprocessing...

Saving gensim dictionary to corpus_data/2018-12-4/Gensim_dict_Params_MWL3_PD0.8__8h12m32s.dict

Saving .mm matrix to corpus_data/2018-12-4/BOWmat_8h12m32s.mm



In [12]:
# Create a new corpus, made of previously unseen documents.

other_corpus = [lda_dict2.doc2bow(text) for text in processed_data2]

lda_model.update(other_corpus)

In [13]:
diag2 = Diagnostics(lda_dict2,
                   bow_rep2, 
                   processed_data2)

warnings.filterwarnings("ignore", category=FutureWarning)

diag.LDAvis(lda_model)

Rendering visualization...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.050564 -0.032944       1        1  63.714771
1     -0.173524 -0.052475       2        1   4.633289
2     -0.029438  0.114261       3        1   4.263703
3     -0.002336  0.043023       4        1   4.509443
4      0.057728 -0.024696       5        1   4.961451
5      0.031171 -0.016015       6        1  10.602460
6      0.065836 -0.031152       7        1   7.314879, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
19    Default  129.000000   oppress  129.000000  30.0000  30.0000
18    Default  187.000000      need  187.000000  29.0000  29.0000
49    Default  112.000000      must  112.000000  28.0000  28.0000
65    Default   75.000000     women   75.000000  27.0000  27.0000
11    Default  130.000000    gender  130.000000  26.0000  26.0000
69    Default  394.000000       art  394.000000  25.0000  25.0000
71    Default  279.000000    commun  279.000000  24.0000  24.0000
57    Default   89.000000     round   89.000000  23.0000  23.0000
112   Default  122.000000      base  122.000000  22.0000  22.0000
46    Default  143.000000     innov  143.000000  21.0000  21.0000
60    Default  135.000000   support  135.000000  20.0000  20.0000
105   Default   79.000000    origin   79.000000  19.0000  19.0000
8     Default   87.000000    divers   87.000000  18.0000  18.0000
37    Default  217.000000    artist  217.000000  17.0000  17.0000
23    Default   46.000000      race   46.000000  16.0000  16.0000
12    Default   72.000000      give   72.000000  15.0000  15.0000
20    Default  129.000000  particip  129.000000  14.0000  14.0000
28    Default  120.000000    social  120.000000  13.0000  13.0000
72    Default  100.000000   creativ  100.000000  12.0000  12.0000
99    Default   48.000000    studio   48.000000  11.0000  11.0000
93    Default  212.000000   program  212.000000  10.0000  10.0000
21    Default  131.000000     peopl  131.000000   9.0000   9.0000
32    Default   50.000000      wide   50.000000   8.0000   8.0000
44    Default   33.000000     earli   33.000000   7.0000   7.0000
263   Default  173.000000     youth  173.000000   6.0000   6.0000
48    Default   94.000000     media   94.000000   5.0000   5.0000
68    Default   38.000000  appropri   38.000000   4.0000   4.0000
24    Default   43.000000      sale   43.000000   3.0000   3.0000
98    Default   94.000000    school   94.000000   2.0000   2.0000
6     Default   81.000000    cultur   81.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
97     Topic7    5.732405      safe   26.265541   1.0931  -5.2686
123    Topic7    3.944788     everi   17.380669   1.1323  -5.6424
50     Topic7    8.592207      open   52.034256   0.8142  -4.8639
72     Topic7   12.864664   creativ  100.868462   0.5559  -4.4603
71     Topic7   21.131855    commun  279.264191   0.0339  -3.9640
10     Topic7    6.397429    experi   45.567581   0.6520  -5.1589
69     Topic7   23.490444       art  394.460266  -0.2057  -3.8582
170    Topic7    9.984822  workshop   97.279709   0.3387  -4.7137
177    Topic7    6.149935      also   48.144417   0.5575  -5.1983
107    Topic7    4.854611     share   34.950081   0.6413  -5.4348
93     Topic7   11.925439   program  212.909515  -0.2669  -4.5361
5      Topic7    7.781748     creat   95.191254   0.1112  -4.9630
60     Topic7    9.205438   support  135.827362  -0.0763  -4.7950
21     Topic7    9.047876     peopl  131.340668  -0.0600  -4.8122
95     Topic7   10.890413    provid  198.455658  -0.2874  -4.6269
94     Topic7    7.276886   project   85.539413   0.1510  -5.0301
37     Topic7   10.711061    artist  217.085068  -0.3938  -4.6435
28     Topic7    7.934198    social  120.979820  -0.1092  -4.9436
181    Topic7    4.877411     bring   42.836609   0.4425  -5.4302
6      Topic7    6.280731    cultur   8

### Again, no improvement with the inclusion of the BC data